# Imports 'n Such 

In [1]:
def fdr(model, x_df, y_df): 
    probs = pd.Series(model.predict_proba(x_df)[:,1]).sort_values(ascending=False)

    numbads = y_df.sum()
    topRows = int(round(len(probs)*0.03))
    detected = y_df[probs.head(topRows).index].sum()
    total = y_df.sum()

    return((detected / total)*100) 

In [2]:
def fdr_nn(model, x_df, y_df): 
    probs = model.predict(x_df)
    probs = pd.DataFrame(probs, columns=['yup'])['yup'].sort_values(ascending=False)

    numbads = y_df.sum()
    topRows = int(round(len(probs)*0.03))
    detected = y_df[probs.head(topRows).index].sum()
    total = y_df.sum()

    return((detected / total)*100) 

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import operator 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import xgboost as xgb 
from keras.models import Sequential
from keras.layers import Dense

import math

In [65]:
mydata = pd.read_csv('CreditCardFraudData_MostCurrent.csv', parse_dates = ['Date'])
#mydata = pd.read_csv('CreditCardFraudData.csv', parse_dates = ['Date'])

# Splitting Data

In [66]:
train_test = mydata[mydata['Date'] <=  '2010-11-01']
oot_X = mydata[mydata['Date'] >  '2010-11-01']

oot_y = oot_X['Fraud']
train_test_Y = train_test['Fraud']



In [68]:
train_test

,Recnum,Date,Fraud,Cardnum,Merchnum,Amount_Cardnum_Merch_description_sum_7,Amount_Merchnum_actual/median_365,Amount_Cardnum_sum_1,Amount_Cardnum_Merch_description_max_7,Amount_Cardnum_Merchnum_sum_30,Amount_Merchnum_max_1,Amount_Merchnum_max_3,Amount_Merchnum_max_14,Amount_Cardnum_Merch zip_median_180,Amount_Cardnum_Merch zip_max_30,Amount_Cardnum_sum_3,Amount_Cardnum_Industry_sum_0,Amount_Cardnum_median_30,Amount_Cardnum_Merch_description_sum_14,Amount_Cardnum_Merchnum_sum_1
0,1,2010-01-01,0,5142190439,5.51E+12,3.62,1.000000,3.62,3.62,3.62,3.62,3.62,3.62,3.620,3.62,3.62,3.62,3.620,3.62,3.62
1,2,2010-01-01,0,5142183973,61003026333,31.42,1.000000,31.42,31.42,31.42,31.42,31.42,31.42,31.420,31.42,31.42,31.42,31.420,31.42,31.42
2,3,2010-01-01,0,5142131721,4.50E+12,178.49,1.000000,178.49,178.49,178.49,178.49,178.49,178.49,178.490,178.49,178.49,178.49,178.490,178.49,178.49
3,4,2010-01-01,0,5142148452,5.51E+12,3.62,1.000000,3.62,3.62,3.62,3.62,3.62,3.62,3.620,3.62,3.62,3.62,3.620,3.62,3.62
4,5,2010-01-01,0,5142190439,5.51E+12,7.24,1.000000,7.24,3.62,7.24,3.62,3.62,3.62,3.620,3.62,7.24,7.24,3.620,7.24,7.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83994,84486,2010-11-01,0,5142121417,9.90E+12,40.01,1.285668,40.01,40.01,40.01,40.01,40.01,382.31,143.475,40.01,40.01,40.01,97.535,40.01,40.01
83995,84487,2010-11-01,0,5142268110,5.90E+12,13.00,1.000000,13.00,13.00,13.00,13.00,13.00,13.00,13.000,13.00,606.32,3281.45,303.160,13.00,13.00
83996,84488,2010-11-01,0,5142124960,6.92E+12,243.62,1.635034,243.62,243.62,243.62,243.62,243.62,243.62,243.620,243.62,243.62,243.62,186.785,243.62,243.62
83997,84489,2010-11-01,0,5142182128,6.05E+12,2325.30,1.000000,2325.30,2325.30,2325.30,2325.30,2325.30,2325.30,2325.300,2325.30,2325.30,2325.30,115.400,2325.30,2325.30


In [69]:
train_test.drop(columns=['Recnum', 'Date', 'Cardnum','Merchnum', 'Fraud'], inplace = True)
oot_X.drop(columns=['Recnum', 'Date', 'Cardnum','Merchnum', 'Fraud'], inplace = True)

C:\Users\drpow\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Modeling

In [9]:
#metric tracking for debugging

#NOTE three nums that get printed at end of run is FDR @3% in order: TRAIN TEST OOT

model_fdr = []

## Logistic Regression 

### No Sampling

In [38]:
x_val = {'train':[],'test':[],'oot':[]} 
for i in range(10): 
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = LogisticRegression(penalty='l1',solver = 'liblinear', C=.1, class_weight = {0:1, 1:10} )
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


1


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


...

C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


7


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


8
9
LogisticRegression(C=0.1, class_weight={0: 1, 1: 10}, penalty='l1',
                   solver='liblinear') 68.07575757575758 65.40909090909092 30.0


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### SMOTE

In [72]:
x_val = {'train':[],'test':[],'oot':[]} 

    
smote = SMOTE(sampling_strategy=.3)
smote.fit_resample(train_test, train_test_Y)
    

for i in range(10): 

    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = LogisticRegression(penalty='l2',C=1,solver = 'liblinear')
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


1


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


...

C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


7


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


8
9
LogisticRegression(C=1, solver='liblinear') 46.27272727272727 45.5 24.07821229050279


C:\Users\drpow\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Random Forest

### No Sampling

In [81]:
x_val = {'train':[],'test':[],'oot':[]} 

for i in range(10): 
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)
    X_train.reset_index(level = 0, drop = True, inplace = True)
    X_test.reset_index(level = 0, drop = True, inplace = True)
    oot_X.reset_index(level = 0, drop = True, inplace = True)
    
    model = RandomForestClassifier(n_estimators = 100, criterion = 'gini', 
                                   max_depth =15, n_jobs=-1, verbose =0, min_samples_split = 200)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))




0
1
2
3
4
5
6
7
8
9
RandomForestClassifier(max_depth=15, min_samples_split=200, n_jobs=-1) 96.98484848484848 84.68181818181817 63.40782122905027


### SMOTE

In [83]:
x_val = {'train':[],'test':[],'oot':[]} 

smote = SMOTE(sampling_strategy=.3)
smote.fit_resample(train_test, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = RandomForestClassifier(n_estimators = 50, criterion = 'gini',
                                  min_samples_split = 300, max_depth = 10, n_jobs=-1, verbose =0)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

0
1
2
3
4
5
6
7
8
9
RandomForestClassifier(max_depth=10, min_samples_split=300, n_estimators=50,
                       n_jobs=-1) 85.25757575757576 80.77272727272728 62.402234636871505


### UnderSampling

In [85]:
x_val = {'train':[],'test':[],'oot':[]} 

RUS = RandomUnderSampler(sampling_strategy=.2)
RUS.fit_resample(train_test, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = RandomForestClassifier(n_estimators = 50, criterion = 'entropy',
                                  min_samples_split = 300, max_depth = 10, n_jobs=-1, verbose =0)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

0
1
2
3
4
5
6
7
8
9
RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=300,
                       n_estimators=50, n_jobs=-1) 93.25757575757575 82.68181818181817 56.14525139664804


## Boosted Trees

### No Sampling

In [93]:
x_val = {'train':[],'test':[],'oot':[]} 

for i in range(10): 
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = xgb.XGBClassifier(n_estimators = 1000, learning_rate  = .03, subsample = 1,
                                 max_depth = 3, n_jobs=-1, verbose = 0)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))




C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:13:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:13:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
[14:13:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but 

C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1
[14:13:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:13:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

7
[14:16:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:16:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


8
[14:16:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:16:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


9
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.03, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbose=0,
              verbosity=None) 96.15151515151516 89.63636363636364 55.754189944134076


### SMOTE

In [92]:
x_val = {'train':[],'test':[],'oot':[]} 

smote = SMOTE(sampling_strategy=.7)
smote.fit_resample(train_test, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = xgb.XGBClassifier(n_estimators = 500, learning_rate  = .05,subsample = 1,
                                 max_depth = 3, n_jobs=-1)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:10:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
[14:10:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1
[14:10:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

7
[14:11:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


8
[14:11:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


9
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None) 95.36363636363635 89.04545454545455 55.418994413407816


### UnderSampling

In [94]:
x_val = {'train':[],'test':[],'oot':[]} 

RUS = RandomUnderSampler(sampling_strategy=.1)
RUS.fit_resample(train_test, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = xgb.XGBClassifier(n_estimators = 100, learning_rate  = .1,subsample = 1,
                                 max_depth = 3, n_jobs=-1)
    model.fit(X_train, y_train)

    x_val['train'].append(fdr(model, X_train, y_train)) 
    x_val['test'].append(fdr(model, X_test, y_test)) 
    x_val['oot'].append(fdr(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:18:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
[14:18:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1
[14:18:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

7
[14:18:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


8
[14:18:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drpow\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


9
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None) 87.98484848484847 83.86363636363636 52.569832402234645


## Neural Net

In [2]:
list(range(100))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

### No Sampling

In [71]:
mms = MinMaxScaler()
train_test_nn = mms.fit_transform(train_test)
oot_X_nn = mms.transform(oot_X)

In [95]:
train_test_nn

array([[1.17730162e-05, 2.46623178e-03, 1.17410573e-05, ...,
        1.35295020e-04, 1.17730162e-05, 1.17730162e-05],
       [1.02435025e-04, 2.46623178e-03, 1.02156956e-04, ...,
        1.22549837e-03, 1.02435025e-04, 1.02435025e-04],
       [5.82063141e-04, 2.46623178e-03, 5.80483078e-04, ...,
        6.99298780e-03, 5.82063141e-04, 5.82063141e-04],
       ...,
       [7.94466617e-04, 4.03247744e-03, 7.92309965e-04, ...,
        7.31828408e-03, 7.94466617e-04, 7.94466617e-04],
       [7.58329001e-03, 2.46623178e-03, 7.56270448e-03, ...,
        4.51885366e-03, 7.58329001e-03, 7.58329001e-03],
       [5.66604943e-04, 1.76546576e-03, 5.88809146e-04, ...,
        7.87515054e-03, 5.66604943e-04, 5.66604943e-04]])

In [99]:
x_val = {'train':[],'test':[],'oot':[]} 

for i in range(10): 
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test_nn, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = Sequential()
    model.add(Dense(10,input_dim =15, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=15, batch_size=10)

    x_val['train'].append(fdr_nn(model, X_train, y_train)) 
    x_val['test'].append(fdr_nn(model, X_test, y_test)) 
    x_val['oot'].append(fdr_nn(model, oot_X_nn, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))




Epoch 1/15
6300/6300 [==============================] - 4s 639us/step - loss: 0.1695 - accuracy: 0.9890
Epoch 2/15
6300/6300 [==============================] - 4s 598us/step - loss: 0.1612 - accuracy: 0.9895
Epoch 3/15
6300/6300 [==============================] - 4s 593us/step - loss: 0.1526 - accuracy: 0.9901
Epoch 4/15
6300/6300 [==============================] - 4s 618us/step - loss: 0.1704 - accuracy: 0.9890
Epoch 5/15
6300/6300 [==============================] - 4s 593us/step - loss: 0.1609 - accuracy: 0.9896
Epoch 6/15
6300/6300 [==============================] - 4s 600us/step - loss: 0.1740 - accuracy: 0.9887
Epoch 7/15
6300/6300 [==============================] - 4s 603us/step - loss: 0.1642 - accuracy: 0.9894
Epoch 8/15
6300/6300 [==============================] - 4s 600us/step - loss: 0.1494 - accuracy: 0.9903
Epoch 9/15
6300/6300 [==============================] - 4s 596us/step - loss: 0.1594 - accuracy: 0.9897
Epoch 10/15
6300/6300 [==============================] - 4s 600u

In [61]:
len(X_train)

24012

In [59]:
oot_X

,Amount_Cardnum_Merch_description_sum_7,Amount_Merchnum_actual/median_365,Amount_Cardnum_sum_1,Amount_Cardnum_Merch_description_max_7,Amount_Cardnum_Merchnum_sum_30,Amount_Merchnum_max_1,Amount_Merchnum_max_3,Amount_Merchnum_max_14,Amount_Cardnum_Merch zip_median_180,Amount_Cardnum_Merch zip_max_30,Amount_Cardnum_sum_3,Amount_Cardnum_Industry_sum_0,Amount_Cardnum_median_30,Amount_Cardnum_Merch_description_sum_14,Amount_Cardnum_Merchnum_sum_1
0,69.64,0.743665,122.61,41.76,27.88,180.16,180.16,509.88,34.820,41.76,122.61,27.88,152.27,69.64,27.88
1,61.37,0.963801,696.37,61.37,61.37,61.37,61.37,61.37,61.370,61.37,696.37,61.37,98.85,61.37,61.37
2,40.00,1.000000,1249.00,40.00,40.00,40.00,40.00,40.00,213.900,392.70,1249.00,40.00,224.50,40.00,40.00
3,51.69,1.000000,119.66,51.69,51.69,51.69,51.69,51.69,51.690,51.69,119.66,51.69,41.65,51.69,51.69
4,109.98,0.746133,109.98,109.98,2916.90,109.98,109.98,1500.00,128.690,1500.00,109.98,109.98,250.00,2916.90,109.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64184,640.00,1.011858,5627.76,640.00,640.00,1458.00,1458.00,3234.00,640.000,640.00,5801.29,3422.17,3.62,640.00,640.00
64185,2560.00,3.014129,7547.76,1920.00,2560.00,1920.00,1920.00,3234.00,1280.000,1920.00,7721.29,5342.17,3.62,2560.00,2560.00
64186,60.10,0.634854,1203.47,45.00,421.79,16.71,16.71,117.94,51.545,330.64,1314.76,60.10,111.29,60.10,15.10
64187,450.00,1.000000,420.00,30.00,450.00,30.00,30.00,30.00,30.000,30.00,420.00,420.00,30.00,450.00,420.00


In [52]:
probs = model.predict(x_df)

In [44]:
probs = model.predict(x_df)
probs = pd.DataFrame(probs, columns=['yup'])['yup'].sort_values(ascending=False)

numbads = y_df.sum()
topRows = int(round(len(probs)*0.03))
detected = y_df[probs.head(topRows).index].sum()
total = y_df.sum()

((detected / total)*100) 

{'train': [36.79653679653679,
  34.1991341991342,
  38.52813852813853,
  39.82683982683983,
  39.82683982683983,
  38.52813852813853,
  41.125541125541126,
  35.4978354978355,
  38.961038961038966,
  34.63203463203463],
 'test': [48.05194805194805,
  40.25974025974026,
  40.25974025974026,
  40.25974025974026,
  42.857142857142854,
  48.05194805194805,
  38.961038961038966,
  35.064935064935064,
  37.66233766233766,
  38.961038961038966],
 'oot': [67.37683089214381,
  66.84420772303595,
  67.11051930758988,
  66.97736351531292,
  67.37683089214381,
  67.37683089214381,
  67.37683089214381,
  67.11051930758988,
  67.11051930758988,
  66.97736351531292]}

### SMOTE

In [102]:
x_val = {'train':[],'test':[],'oot':[]} 

smote = SMOTE(sampling_strategy=.3)
smote.fit_resample(train_test_nn, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test_nn, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)

    model = Sequential()
    model.add(Dense(8,input_dim =15, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=15, batch_size=10)


    x_val['train'].append(fdr_nn(model, X_train, y_train)) 
    x_val['test'].append(fdr_nn(model, X_test, y_test)) 
    x_val['oot'].append(fdr_nn(model, oot_X_nn, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

Epoch 1/15
6300/6300 [==============================] - 4s 611us/step - loss: 0.1043 - accuracy: 0.9898
Epoch 2/15
6300/6300 [==============================] - 4s 590us/step - loss: 0.0323 - accuracy: 0.9918
Epoch 3/15
6300/6300 [==============================] - 4s 622us/step - loss: 0.0303 - accuracy: 0.9924
Epoch 4/15
6300/6300 [==============================] - 4s 707us/step - loss: 0.0308 - accuracy: 0.9926
Epoch 5/15
6300/6300 [==============================] - 4s 692us/step - loss: 0.0301 - accuracy: 0.9926
Epoch 6/15
6300/6300 [==============================] - 4s 705us/step - loss: 0.0310 - accuracy: 0.9921
Epoch 7/15
6300/6300 [==============================] - 4s 692us/step - loss: 0.0286 - accuracy: 0.9933
Epoch 8/15
6300/6300 [==============================] - 4s 673us/step - loss: 0.0293 - accuracy: 0.9927
Epoch 9/15
6300/6300 [==============================] - 4s 678us/step - loss: 0.0288 - accuracy: 0.9929
Epoch 10/15
6300/6300 [==============================] - 4s 681u

### UnderSampling

In [45]:
x_val = {'train':[],'test':[],'oot':[]} 

RUS = RandomUnderSampler(sampling_strategy=.3)
RUS.fit_resample(train_test, train_test_Y)

for i in range(10): 

    
    X_train, X_test, y_train, y_test =\
    train_test_split(train_test, train_test_Y,
    test_size=0.25,
    stratify=train_test_Y)

    y_train.reset_index(level = 0, drop = True, inplace = True)
    y_test.reset_index(level = 0, drop = True, inplace = True)
    oot_y.reset_index(level = 0, drop = True, inplace = True)
    
    model = Sequential()
    model.add(Dense(12,input_dim =14, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=6, batch_size=10)

    x_val['train'].append(fdr_nn(model, X_train, y_train)) 
    x_val['test'].append(fdr_nn(model, X_test, y_test)) 
    x_val['oot'].append(fdr_nn(model, oot_X, oot_y)) 
    print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['test']), np.mean(x_val['oot']))

Epoch 1/6
6300/6300 [==============================] - 5s 768us/step - loss: 3.7892 - accuracy: 0.96620s - loss:
Epoch 2/6
6300/6300 [==============================] - 5s 760us/step - loss: 0.0627 - accuracy: 0.9895
Epoch 3/6
6300/6300 [==============================] - 5s 795us/step - loss: 0.0609 - accuracy: 0.9894
Epoch 4/6
6300/6300 [==============================] - 5s 804us/step - loss: 0.0758 - accuracy: 0.9883
Epoch 5/6
6300/6300 [==============================] - 5s 798us/step - loss: 0.0583 - accuracy: 0.9898
Epoch 6/6
6300/6300 [==============================] - 5s 830us/step - loss: 0.0650 - accuracy: 0.9892
0
Epoch 1/6
6300/6300 [==============================] - 5s 808us/step - loss: 4.9329 - accuracy: 0.9052
Epoch 2/6
6300/6300 [==============================] - 5s 795us/step - loss: 0.0718 - accuracy: 0.9892
Epoch 3/6
6300/6300 [==============================] - 5s 797us/step - loss: 0.0612 - accuracy: 0.9901
Epoch 4/6
6300/6300 [==============================] - 5s 784

In [54]:
model.evaluate(test_train)

382/382 [==============================] - 0s 473us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

## Final Model

In [111]:
X_final = train_test
y_final = train_test_Y
# oot_X
# oot_y

In [105]:
x_val = {'train':[],'oot':[]} 
smote = SMOTE(sampling_strategy=.3)
smote.fit_resample(train_test, train_test_Y)

#     X_train, X_test, y_train, y_test =\
#     train_test_split(train_test, train_test_Y,
#     test_size=0.25,
#     stratify=train_test_Y)

#     y_train.reset_index(level = 0, drop 1= True, inplace = True)
#     y_test.reset_index(level = 0, drop = True, inplace = True)
oot_y.reset_index(level = 0, drop = True, inplace = True)

model = RandomForestClassifier(n_estimators = 50, criterion = 'gini',
                              min_samples_split = 300, max_depth =10, n_jobs=-1, verbose =0)
model.fit(X_final, y_final)

x_val['train'].append(fdr(model, X_final, y_final)) 
x_val['oot'].append(fdr(model, oot_X, oot_y)) 
print(i)

model_fdr.append([model, np.mean(x_val['train']), np.mean(x_val['oot'])])

print(model, np.mean(x_val['train']), np.mean(x_val['oot']))



0
1
2
3
4
5
6
7
8
9
RandomForestClassifier(max_depth=10, min_samples_split=300, n_estimators=50,
                       n_jobs=-1) 84.55681818181819 62.17877094972067


In [112]:
x_meas = oot_X
y_meas = oot_y

In [113]:
probs = model.predict_proba(x_meas)
probs = pd.DataFrame(probs, columns=['nope','predict_fraud_prob'])
probs['predict'] = model.predict(x_meas)
probs['actual_fraud_label'] = y_meas
probs.drop(columns=['nope'], inplace=True)
probs.sort_values('predict_fraud_prob', ascending =False, inplace=True)
probs.to_csv('OOT.csv')

In [110]:
y_meas.sum()

880